**VIEW**

ساخت یک ویو برای محاسبه فروش به تفکیک سال، فصل و ماه

In [ ]:
DROP VIEW IF EXISTS Sales.vwMonthlySalesOrder
GO

CREATE VIEW Sales.vwMonthlySalesOrder
AS
SELECT 
	YEAR(SOH.OrderDate)					AS OrderYear,
	DATEPART(QUARTER, SOH.OrderDate)	AS OrderQuarter,
	MONTH(SOH.OrderDate)				AS OrderMonth,
	SUM(SOD.OrderQty)					AS TotalOrderQty,
	SUM(SOD.OrderQty * SOD.UnitPrice)	AS TotalPrice
FROM
	Sales.SalesOrderHeader AS SOH --31,465
INNER JOIN
	Sales.SalesOrderDetail AS SOD --121,317
ON
	SOH.SalesOrderID = SOD.SalesOrderID
GROUP BY
	YEAR(SOH.OrderDate),
	DATEPART(QUARTER, SOH.OrderDate),
	MONTH(SOH.OrderDate)

از ویو می توان مانند یک جدول کویری گرفت

In [ ]:
SELECT TOP 10
	*
FROM
	Sales.vwMonthlySalesOrder
WHERE
	OrderYear > 2011
ORDER BY
	OrderMonth

مساله:

**فروش به تفکیک سال و فصل**

**به صورت ماترسی**

  

روش اول:  

VIEW + JOIN

ابتدا ویو ها را می سازیم

In [ ]:
-- VIEW: 
-- لیست سال ها
DROP VIEW IF EXISTS Sales.vwSales_Years
GO

CREATE VIEW Sales.vwSales_Years
AS
SELECT 
	YEAR(SOH.OrderDate)					AS OrderYear
FROM
	Sales.SalesOrderHeader AS SOH 
GROUP BY
	YEAR(SOH.OrderDate)

In [ ]:
-- VIEW:
-- فروش فصل اول به تفکیک سال

DROP VIEW IF EXISTS Sales.vwSales_Q1
GO

CREATE VIEW Sales.vwSales_Q1
AS
SELECT 
	YEAR(SOH.OrderDate)					AS OrderYear,
	SUM(SOD.OrderQty * SOD.UnitPrice)	AS Q1_Sales
FROM
	Sales.SalesOrderHeader AS SOH --31,465
INNER JOIN
	Sales.SalesOrderDetail AS SOD --121,317
ON
	SOH.SalesOrderID = SOD.SalesOrderID
WHERE
	DATEPART(QUARTER, SOH.OrderDate) = 1
GROUP BY
	YEAR(SOH.OrderDate)

In [ ]:
-- VIEW:
-- فروش فصل دوم به تفکیک سال

DROP VIEW IF EXISTS Sales.vwSales_Q2
GO

CREATE VIEW Sales.vwSales_Q2
AS
SELECT 
	YEAR(SOH.OrderDate)					AS OrderYear,
	SUM(SOD.OrderQty * SOD.UnitPrice)	AS Q2_Sales
FROM
	Sales.SalesOrderHeader AS SOH --31,465
INNER JOIN
	Sales.SalesOrderDetail AS SOD --121,317
ON
	SOH.SalesOrderID = SOD.SalesOrderID
WHERE
	DATEPART(QUARTER, SOH.OrderDate) = 2
GROUP BY
	YEAR(SOH.OrderDate)

In [ ]:
-- VIEW:
-- فروش فصل سوم به تفکیک سال
DROP VIEW IF EXISTS Sales.vwSales_Q3
GO

CREATE VIEW Sales.vwSales_Q3
AS
SELECT 
	YEAR(SOH.OrderDate)					AS OrderYear,
	SUM(SOD.OrderQty * SOD.UnitPrice)	AS Q3_Sales
FROM
	Sales.SalesOrderHeader AS SOH --31,465
INNER JOIN
	Sales.SalesOrderDetail AS SOD --121,317
ON
	SOH.SalesOrderID = SOD.SalesOrderID
WHERE
	DATEPART(QUARTER, SOH.OrderDate) = 3
GROUP BY
	YEAR(SOH.OrderDate)

In [ ]:
-- VIEW:
-- فروش فصل چهارم به تفکیک سال

DROP VIEW IF EXISTS Sales.vwSales_Q4
GO

CREATE VIEW Sales.vwSales_Q4
AS
SELECT 
	YEAR(SOH.OrderDate)					AS OrderYear,
	SUM(SOD.OrderQty * SOD.UnitPrice)	AS Q4_Sales
FROM
	Sales.SalesOrderHeader AS SOH --31,465
INNER JOIN
	Sales.SalesOrderDetail AS SOD --121,317
ON
	SOH.SalesOrderID = SOD.SalesOrderID
WHERE
	DATEPART(QUARTER, SOH.OrderDate) = 4
GROUP BY
	YEAR(SOH.OrderDate)

In [ ]:
پس از ساخت ویو، آن ها را با جوین کنار هم قرار می دهیم

In [ ]:
SELECT 
	Y.OrderYear,
	Q1.Q1_Sales,
	Q2.Q2_Sales,
	Q3.Q3_Sales,
	Q4.Q4_Sales
FROM 
	Sales.vwSales_Years AS Y
LEFT OUTER JOIN
	Sales.vwSales_Q1 AS Q1
ON
	Y.OrderYear = Q1.OrderYear
LEFT OUTER JOIN
	Sales.vwSales_Q2 AS Q2
ON
	Y.OrderYear = Q2.OrderYear
LEFT OUTER JOIN
	Sales.vwSales_Q3 AS Q3
ON
	Y.OrderYear = Q3.OrderYear
LEFT OUTER JOIN
	Sales.vwSales_Q4 AS Q4
ON
	Y.OrderYear = Q4.OrderYear
ORDER BY
	Y.OrderYear